In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle
import json
from tqdm import tqdm
import os

import nomad.io.base as loader
import nomad.city_gen as cg
import nomad.traj_gen as tg
from nomad.traj_gen import Agent, Population
from nomad.city_gen import City

import nomad.data as data_folder
from pathlib import Path
data_dir = Path(data_folder.__file__).parent
path = data_dir / "garden_city.gpkg"

In [2]:
heatmap_df = loader.from_file('sparse_traj_2/part-0.parquet', format='parquet')

FileNotFoundError: sparse_traj_2/part-0.parquet